In [3]:
# utility functions for accessing transfermarkt.com

from urllib2 import urlopen, Request
from bs4 import BeautifulSoup
from pandas import DataFrame

useragent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
urlprefix = 'http://www.transfermarkt.com/lol/'

def getUrlFromLeagueId(id, season=2015):
    return urlprefix + 'startseite/wettbewerb/' + id + '/saison_id/' + str(season)

def getClubsFromLeagueId(id, season=2015):
    bs = BeautifulSoup(urlopen(Request(getUrlFromLeagueId(id, season), headers={'User-Agent': useragent})))
    elements = bs.find(id='yw1').find_all("td",class_="hauptlink no-border-links hide-for-small hide-for-pad")
    return DataFrame([{'id': cp.find("a")["id"], 'name': cp.getText(), 'url': cp.find("a")["href"]} for cp in elements])

def getUrlFromClubId(id, season=2015):
    return urlprefix + 'startseite/verein/' + str(id) + '/saison_id/' + str(season)

def getPlayersFromClubId(id, season=2015):
    bs = BeautifulSoup(urlopen(Request(getUrlFromClubId(418), headers={'User-Agent': useragent})))
    elements = bs.find(id='yw1').find_all("span",class_="hide-for-small")
    return DataFrame([{'id': cp.find("a", class_="spielprofil_tooltip")["id"], 'name': cp.getText(), 'url': cp.find("a")["href"]} for cp in elements 
                      if cp.find("a", class_="spielprofil_tooltip")])

def getUrlFromPlayerId(id):
    return urlprefix + 'profil/spieler/' + str(id)

In [8]:
getClubsFromLeagueId('GB1',2010)

,id,name,url
0,631,Chelsea FC,/chelsea-fc/startseite/verein/631/saison_id/2010
1,985,Manchester United,/manchester-united/startseite/verein/985/saiso...
2,281,Manchester City,/manchester-city/startseite/verein/281/saison_...
3,31,Liverpool FC,/liverpool-fc/startseite/verein/31/saison_id/2010
4,11,Arsenal FC,/arsenal-fc/startseite/verein/11/saison_id/2010
5,148,Tottenham Hotspur,/tottenham-hotspur/startseite/verein/148/saiso...
6,405,Aston Villa,/aston-villa/startseite/verein/405/saison_id/2010
7,29,Everton FC,/everton-fc/startseite/verein/29/saison_id/2010
8,379,West Ham United,/west-ham-united/startseite/verein/379/saison_...
9,289,Sunderland AFC,/sunderland-afc/startseite/verein/289/saison_i...


In [9]:
getPlayersFromClubId(418,2005)

,id,name,url
0,79422,Keylor Navas,/keylor-navas/profil/spieler/79422
1,27486,Kiko Casilla,/kiko-casilla/profil/spieler/27486
2,246335,Rubén Yáñez,/ruben-yanez/profil/spieler/246335
3,25557,Sergio Ramos,/sergio-ramos/profil/spieler/25557
4,164770,Raphaël Varane,/raphael-varane/profil/spieler/164770
5,14132,Pepe,/pepe/profil/spieler/14132
6,58884,Nacho Fernández,/nacho-fernandez/profil/spieler/58884
7,44501,Marcelo,/marcelo/profil/spieler/44501
8,138927,Daniel Carvajal,/daniel-carvajal/profil/spieler/138927
9,145707,Danilo,/danilo/profil/spieler/145707
